In [1]:
%run ./frocc.ipynb

In [34]:
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

In [16]:
# https://osf.io/jda6s/
mnist = loadmat("datasets/mnist-original.mat")

In [57]:
digits = mnist['data'].T
labels = mnist['label'].flatten()

In [58]:
# only 0 is outlier
labels = np.array(list(map(lambda x: 0 if x != 0 else 1, labels)))

In [60]:
x_train, x_test, y_train, y_test = train_test_split(
  digits, labels, test_size=0.3
)

In [61]:
train_inliers = np.argwhere(y_train == 0).flatten()
x_train = x_train[train_inliers]
y_train = y_train[train_inliers]

In [76]:
def run(m, eps, output=True):
  d = x_train.shape[1]

  detector = frocc(m, eps, d)
  detector.train(x_train)
  y_pred = detector.test(x_test)

  tp, fn, fp, tn = confusion_matrix(y_test, y_pred).ravel()
  tpr = tp/(tp + fn)
  fpr = fp/(fp + tn)
  acc = (tp + tn)/(tp + fn + fp + tn)

  if output:
    print("TP:  {} \t FN:  {}".format(tp, fn))
    print("FP:  {} \t TN:  {}".format(fp, tn))
    print("TPR: {:.3f} \t FPR: {:.3f}".format(tpr, fpr))
    print("ACC: {:.3f} \t LR+: {:.3f}".format(acc, tpr/fpr))

In [84]:
m = 20
eps = 0.0002
run(m, eps)

TP:  12685 	 FN:  6195
FP:  631 	 TN:  1489
TPR: 0.672 	 FPR: 0.298
ACC: 0.675 	 LR+: 2.257
